In [1]:
from keras.models import Sequential
from keras.layers import Dense,Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys

In [2]:
f= open('../datasets/shakespeare.txt','r')
d=str(f.read())
f.close()

In [3]:
data= d.strip().lower()
data=data[104:]

In [10]:
print data[5000:5100]

 a refund of the money (if any) you paid
for it by sending an explanatory note within that time to 


In [5]:
chars=sorted(list(set(data)))
print len(chars)

66


In [7]:
char_indices=dict((c,i) for i,c in enumerate(chars))
indices_char=dict((i,c) for i,c in enumerate(chars))
print len(char_indices),len(indices_char)

66 66


In [15]:
sent=data.split('\n')

In [25]:
lengths=[]
for ix in sent:
    lengths.append(len(ix))
print len(lengths)

124784


In [26]:
uniq, nums= np.unique(np.array(lengths),return_counts=True)

In [28]:
print uniq[:10],nums[:10]

 [ 1  5  6  7  8  9 10 11 12 13] [9681   43   77   33  103  264  451  407  278  273]


In [30]:
maxlen=50
step=3
sentences=[]
next_chars=[]
for i in range(0,len(data)-maxlen,step):
    sentences.append(data[i:i+maxlen])
    next_chars.append(data[i+maxlen])
print ('nb sequences: ' , len(sentences))

('nb sequences: ', 1863245)


In [34]:
print sentences[100:110]

['ook, details below **\r\n**     please follow the co', ', details below **\r\n**     please follow the copyr', 'etails below **\r\n**     please follow the copyrigh', 'ils below **\r\n**     please follow the copyright g', ' below **\r\n**     please follow the copyright guid', 'low **\r\n**     please follow the copyright guideli', ' **\r\n**     please follow the copyright guidelines', '\r\n**     please follow the copyright guidelines in', '*     please follow the copyright guidelines in th', '   please follow the copyright guidelines in this ']


In [36]:
print ('Vectorization...')
x= np.zeros((len(sentences),maxlen,len(chars)),dtype=np.bool)
y=np.zeros((len(sentences),len(chars)),dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i,t, char_indices[char]]=1
    y[i,char_indices[next_chars[i]]]=1

Vectorization...


In [37]:
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

model.summary()

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lstm_2 (LSTM)                    (None, 128)           99840       lstm_input_2[0][0]               
____________________________________________________________________________________________________
dense_2 (Dense)                  (None, 66)            8514        lstm_2[0][0]                     
____________________________________________________________________________________________________
activation_2 (Activation)        (None, 66)            0           dense_2[0][0]                    
Total params: 108,354
Trainable params: 108,354
Non-trainable params: 0
____________________________________________________________________________________________________


In [38]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return probas
    # return np.argmax(probas)

In [39]:
prob = np.array([0.2, 0.1, 0.4, 0.15, 0.05, 0.1])

a = sample(prob, temperature=1.1)

print prob
print a

#plt.plot(prob)
#plt.plot(a, 'r--*')

[ 0.2   0.1   0.4   0.15  0.05  0.1 ]
[[0 0 1 0 0 0]]


In [42]:
for iteration in range(1, 10):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x, y, shuffle=True,
              batch_size=128,
              nb_epoch=1, verbose=2)

    start_index = random.randint(0, len(text) - maxlen - 1)

    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            xx = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.

            preds = model.predict(xx, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

()
--------------------------------------------------
('Iteration', 1)
Epoch 1/1


KeyboardInterrupt: 